# Dynamic Programming 

----

## 1. Rob Cutting Problems

![](https://github.com/pchen12567/picture_store/blob/master/AI_For_NLP/dynamic_p1.png?raw=true)

In [1]:
from collections import defaultdict

In [2]:
# Set each length price
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]
price = defaultdict(int)

In [3]:
for i, p in enumerate(original_price):
    price[i + 1] = p

In [4]:
print(price.items())

dict_items([(1, 1), (2, 5), (3, 8), (4, 9), (5, 10), (6, 17), (7, 17), (8, 20), (9, 24), (10, 30)])


### Try to build solution by recursive

![](https://github.com/pchen12567/picture_store/blob/master/AI_For_NLP/dynamic_p2.jpg?raw=true)

In [5]:
# Build solution by recursive
def revenue(length):
    return max(
        [price[length]] + [revenue(i) + revenue(length - i) for i in range(1, length)]
    )        

In [6]:
revenue(15)

43

#### Detail about function max() and lambda

In [7]:
import random

In [8]:
random.seed(0)
random_numbers = [(i, random.randint(-10,20)) for i in range(10)]
print(random_numbers)

[(0, 17), (1, 2), (2, 14), (3, 18), (4, 3), (5, -9), (6, -2), (7, 20), (8, 6), (9, 5)]


In [9]:
max(random_numbers, key=lambda x: x[1])

(7, 20)

### Optimize function to get solution

In [10]:
# Init solution dictionary
solution = {}
# for a given length N, we set the corresponding split parts
# solution = {
#     4: (2, 2)
# }

In [11]:
def optimize_revenue(length):
    max_price, split_point = max(
        [(price[length], 0)] +
        [(optimize_revenue(i) + optimize_revenue(length - i), i) for i in range(1, length)], key=lambda x: x[0]
    )
    solution[length] = (split_point, length - split_point)
    
    return max_price

In [12]:
optimize_revenue(15)

43

In [13]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10),
 13: (3, 10),
 14: (2, 12),
 15: (2, 13)}

### Build function to parse solution

Check the solution step by step

In [14]:
# Step 1
solution[15] # cut to 2 and 16

(2, 13)

In [15]:
# Step 2
solution[2] # no cut, keep 2

(0, 2)

In [16]:
# Step 3
solution[13] # cut to 3 and 10

(3, 10)

In [17]:
# Step 4
solution[3] # not cut, keep 3

(0, 3)

In [18]:
# Step 5
solution[10] # not cut, keep 10

(0, 10)

In [19]:
def parse_solution(length, solution):
    left, right = solution[length]
    
    if left == 0: return [right]
    
    return [left] + parse_solution(right, solution)

In [20]:
parse_solution(15, solution)

[2, 3, 10]

### Decorator

#### Detail about decorator

In [21]:
import time

In [22]:
def call_time(func, arg):
    start = time.time()
    func(arg)
    print('New feature, {} Cost time: {}'.format(func.__name__, time.time() - start))

In [23]:
def func_normal(n):
    for i in range(n):
        print(i)

In [24]:
func_normal(5)

0
1
2
3
4


In [25]:
call_time(func_normal, 5)

0
1
2
3
4
New feature, func_normal Cost time: 0.002274036407470703


In [26]:
def func_slow(n):
    for i in range(n):
        time.sleep(0.5)
        print(i)

In [27]:
func_slow(5)

0
1
2
3
4


In [28]:
call_time(func_slow, 5)

0
1
2
3
4
New feature, func_slow Cost time: 2.514373302459717


In [29]:
def get_call_time(func):
    def _wrap(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print('New feature, {} Cost time: {}'.format(func.__name__, time.time() - start))
        return result
    return _wrap

In [30]:
func_wrap = get_call_time(func_normal) # => @ decorator
print(func_wrap)

<function get_call_time.<locals>._wrap at 0x111dab378>


In [31]:
func_wrap(5)

0
1
2
3
4
New feature, func_normal Cost time: 0.0002980232238769531


**With decorator**

In [32]:
@get_call_time
def func_1(n):
    for i in range(n):
        print(i)

func_1(5)

0
1
2
3
4
New feature, func_1 Cost time: 0.00026106834411621094


Equals:<br>
`func_1 = get_call_time(func_1) => func_1 = _wrap(*args, **kwargs)`<br>
`func_1(5) => _wrap(5)`

**Without decorator**

In [33]:
# @get_call_time
def func_1(n):
    for i in range(n):
        print(i)

func_1(5)

0
1
2
3
4


#### Implement decorator

In [34]:
from functools import wraps

In [35]:
def memo(func):
    cache = {}
    @wraps(func)
    def _wrap(*args, **kwargs):
        str_param = str(args) + str(kwargs)
        
        if str_param not in cache:
            result = func(*args, **kwargs)
            cache[str_param] = result
            
        return cache[str_param]
    
    return _wrap

In [36]:
@memo
def optimize_revenue(length):
    max_price, split_point = max(
        [(price[length], 0)] +
        [(optimize_revenue(i) + optimize_revenue(length - i), i) for i in range(1, length)], key=lambda x: x[0]
    )
    solution[length] = (split_point, length - split_point)
    
    return max_price

In [37]:
optimize_revenue(73)

218

In [38]:
parse_solution(73, solution)

[3, 10, 10, 10, 10, 10, 10, 10]

### Dynamic Programming Summary

不断查表的意思

- 分析子问题的重复性
- 子问题进行存储
- Solution 要进行解析

----

## 2. Edit Distance

编辑距离的定义是：从字符串A到字符串B，中间需要的最少操作权重。这里的操作权重一般是：

- 删除一个字符(deletion)： 删除A末尾一个字符
- 插入一个字符(insertion)： 将B末尾一个字符插入到A末尾
- 替换一个字符(substitution)： 将A末尾的一个字符替换成B末尾的一个字符

- 他们的权重都是1

### Min Edit Distance Definition (Levenshtein)

Given two words word1 and word2, find the minimum number of steps required to convert word1 to word2. 

Having the following 3 operations permitted on a word:

1. Insertion => cost = 1
2. Deletion => cost = 1
3. Substitution => cost = 2

### Min Edit Distance Algorithm

- For two strings
    - X of length n
    - Y of length m 
<br><br>
- We define D(i, j)
    - The edit distance between X[1…i] and Y[1..j]
<br><br>
- The edit distance between X and Y is thus D(n, m)

![](https://github.com/pchen12567/picture_store/blob/master/AI_For_NLP/dynamic_p3.jpg?raw=true)

In [39]:
@memo
def get_edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    return min(
        [get_edit_distance(string1[:-1], string2) + 1, 
         get_edit_distance(string1, string2[:-1]) + 1, 
         get_edit_distance(string1[:-1], string2[:-1]) + (0 if string1[-1] == string2[-1] else 2)]
    )

In [40]:
get_edit_distance('biejing', 'beijing') # Del Ins

2

In [41]:
get_edit_distance('biejing', 'beijie') # Del Ins Sub Del

5

In [42]:
get_edit_distance('biejing', 'beijin') # Del Ins Del

3

# Solution Parse